In [ ]:
# pep.py

import time
import logging
import psutil
import os
import uuid
import requests
import csv
from flask import Flask, request, jsonify

app = Flask(__name__)

class PEP:
    def __init__(self):
        self.setup_logging()
        self.setup_csv_logging()

    def setup_logging(self):
        # Configure the logging module
        logging.basicConfig(
            filename='pep.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )

    def setup_csv_logging(self):
        # Open the CSV file in append mode
        self.csv_file = open('pep_requests.csv', 'a', newline='')
        self.csv_writer = csv.writer(self.csv_file)
        # Write headers if the file is empty
        if os.stat('pep_requests.csv').st_size == 0:
            self.csv_writer.writerow(['Request ID', 'Timestamp', 'Sender', 'Destination Entity', 'Processing Time (s)', 'Total CPU Time (s)'])

    def handle_request(self, entity_id, resource_url, request_data, request_id):
        """
        Handles incoming requests by forwarding them to the destination resource and acknowledging the requester.
        """
        # Log the request receipt
        logging.info(f"Request ID: {request_id} | PEP: Received request from '{entity_id}' for resource '{resource_url}'")

        # Forward the request to the destination resource
        access_decision = self.forward_request(entity_id, resource_url, request_data, request_id)

        return access_decision

    def forward_request(self, entity_id, resource_url, request_data, request_id):
        """
        Forwards the request to the destination resource and returns the response status.
        """
        try:
            # Forward the request to the destination resource
            response = requests.post(resource_url, json=request_data)
            response.raise_for_status()
            logging.info(f"Request ID: {request_id} | PEP: Successfully forwarded request to '{resource_url}'")
            # Acknowledge the requester
            return 'success'
        except Exception as e:
            logging.error(f"Request ID: {request_id} | PEP Error: Failed to forward request to '{resource_url}': {e}")
            return 'failure'

    def log_request(self, request_id, sender, destination, processing_time, total_cpu_time):
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        self.csv_writer.writerow([request_id, timestamp, sender, destination, f"{processing_time:.6f}", f"{total_cpu_time:.6f}"])
        self.csv_file.flush()

    def __del__(self):
        self.csv_file.close()

# Flask API Endpoints

PEP_instance = PEP()  # Create an instance of PEP

@app.route('/handle_request', methods=['POST'])
def handle_request_endpoint():
    try:
        json_data = request.get_json()
        entity_id = json_data.get('entity_id')
        resource_url = json_data.get('resource_url')  # Updated to accept resource URL
        request_data = json_data.get('request_data', {})
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        # Call PEP's handle_request method
        access_decision = PEP_instance.handle_request(entity_id, resource_url, request_data, request_id)

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data and write to CSV
        PEP_instance.log_request(request_id, entity_id, resource_url, processing_time, total_cpu_time)

        response = {
            'request_id': request_id,
            'status': access_decision
        }
        return jsonify(response), 200 if access_decision == 'success' else 500
    except Exception as e:
        logging.error(f"Error in handle_request_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Run the Flask app on all interfaces, port 5000
    app.run(host='0.0.0.0', port=5000)